---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [5]:
# for G in P1_Graphs:
#     print(nx.average_clustering(G), nx.average_shortest_path_length(G), len(degree_distribution(G)))

0.03167539146454044 4.099161161161161 29
0.5642419635919628 5.089871871871872 7
0.4018222222222227 9.378702269692925 5
0.03780379975223251 3.1048046283934134 37
0.0033037037037037037 5.0785509568313305 9


In [3]:
def graph_identification():
    
    # Your Code Here    
    def degree_distribution(G):
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i) / float(nx.number_of_nodes( G)) 
                     for i in degree_values]
        return histogram

    algorithm = []
    for G in P1_Graphs:
        clustering = nx.average_clustering(G)
        shortest_path = nx.average_shortest_path_length(G)
        degree_hist = degree_distribution(G)
        if len(degree_hist)>10:
            algorithm.append('PA')
        elif clustering < 0.1:
            algorithm.append('SW_H')
        else:
            algorithm.append('SW_L')
    return algorithm

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [8]:
# df.head()

,clustering,degree,degree_centrality,closeness,betweeness,pr,is_management
0,0.276423,44,0.043825,0.421991,0.001124,0.001224,0.0
1,0.265306,52,0.051793,0.422360,0.001195,0.001426,NaN
2,0.297803,95,0.094622,0.461490,0.006570,0.002605,NaN
3,0.384910,71,0.070717,0.441663,0.001654,0.001833,1.0
4,0.318691,96,0.095618,0.462152,0.005547,0.002526,1.0


In [9]:
# Import libraries
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

def salary_predictions():
    # Your Code Here
    def is_management(node):
        managementSalary = node[1]['ManagementSalary']
        if managementSalary == 0:
            return 0
        elif managementSalary == 1:
            return 1
        else:
            return None

    # Creating a matrix(df) of node features using networkx
    df = pd.DataFrame(index=G.nodes())
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))
    df['is_management'] = pd.Series([is_management(node) for node in G.nodes(data=True)])
    
    # Split the df to train and test datasets
    # based on is_management column (target)
    df_train = df[~pd.isnull(df['is_management'])] # pd.notnull
    df_test = df[pd.isnull(df['is_management'])]
    
    # Select X_train, y_train, X_test
    features = ['clustering', 'degree', 'degree_centrality', 'closeness', 'betweeness', 'pr']
    X_train = df_train[features]
    y_train = df_train['is_management']
    X_test = df_test[features]
    

    # Make X_train, X_test on the same scale
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Neural Networks model
    clf =  MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5, random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, y_train)
#     test_proba = clf.predict(X_test_scaled)
#     return pd.Series(test_proba,X_test.index)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    return pd.Series(test_proba,X_test.index)

In [6]:
# salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [10]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head()

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [20]:
# df.head()

,Future Connection,preferential_attachment,cn_soundarajan_hopcroft,resource_allocation_index,jaccard_coefficient
"(0, 2)",0.0,4180,6,0.055340,0.045802
"(0, 3)",0.0,3124,3,0.021388,0.027273
"(0, 4)",0.0,4224,3,0.021388,0.022222
"(0, 7)",0.0,3168,4,0.061668,0.036364
"(0, 8)",0.0,1628,1,0.011628,0.012821


In [11]:
# Import libraries
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
  
def new_connections_predictions():
    # Your Code Here
    # Compute the preferential attachment score of all node pairs in ebunch. as df
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]
    
    # Count the number of common neighbors of all node pairs in ebunch. as df_cn
    cn_soundarajan_hopcroft = list(nx.cn_soundarajan_hopcroft(G, community='Department'))
    df_cn = pd.DataFrame(index=[(x[0], x[1]) for x in cn_soundarajan_hopcroft])
    df_cn['cn_soundarajan_hopcroft'] = [x[2] for x in cn_soundarajan_hopcroft]
    
    # Join df with df_cn
    df = df.join(df_cn,how='outer')
    # add more features: resource_allocation_index, jaccard_coefficient
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(value=0)
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    # Join the previous df with the future_connections
    df = future_connections.join(df,how='outer')
    
    # Split the df to train and test datasets
    # based on Future Connection column
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
    
    # Select X_train, y_train, X_test
    features = ['cn_soundarajan_hopcroft', 'preferential_attachment', 'resource_allocation_index', 'jaccard_coefficient']
    X_train = df_train[features]
    Y_train = df_train['Future Connection']
    X_test = df_test[features]
        
    # Make X_train, X_test on the same scale
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Gradient Boosting model
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5, random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    
    # Your predictions will need to be given as the probability
    # of the corresponding edge being a future connection.
    predictions = pd.Series(test_proba,X_test.index)
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['prob'] = [predictions[x] for x in target.index]
    return target['prob']
#     predictions = pd.Series(test_proba,X_test.index)
#     target = future_connections[pd.isnull(future_connections['Future Connection'])]
#     return [predictions[x] for x in target.index]

In [16]:
# new_connections_predictions()

In [ ]:
# # provide a checksum on the index
# # remove this before submission as the grader does not allow to print
# import numpy as np
# nc = new_connections_predictions()
# print("sum 1st nodes", np.sum(list(map(lambda e:e[0], nc.index.values))))
# print("sum 2nd nodes", np.sum(list(map(lambda e:e[1], nc.index.values))))
# print("sum all nodes", np.sum(list(map(np.sum, nc.index.values))))